# Unfolded Map SDK with Data SDK and Bokeh
This example illustrates how to fetch the data that is published on our map and with our Data SDK visualize it on a chart with [Bokeh library](https://docs.bokeh.org/en/latest/index.html)

This example is using the beta [unfolded.map-sdk](https://pypi.org/project/unfolded.map-sdk/) module in Jupyter to enable map loading

In [1]:
from unfolded.map_sdk import UnfoldedMap

First we create a map using a UUID of a published map:

In [2]:
my_map = UnfoldedMap(mapUUID='80c800cc-5805-4416-94a5-bd8105cab7f7')

We can use the [`jupyterlab-sidecar`](https://github.com/jupyter-widgets/jupyterlab-sidecar) package to display the map in JupyterLab's sidebar.
You can install the package with `pip install sidecar`

In [3]:
from sidecar import Sidecar

sc = Sidecar(title='Unfolded Map', anchor='split-right')
with sc:
    display(my_map)

By using Unfolded's [unfolded.data_sdk](https://pypi.org/project/unfolded.data-sdk/) module with your personal [refresh token](https://studio.unfolded.ai/home/tokens) you have access to our [DataSDK](https://docs.unfolded.ai/data-sdk/use-with-python)

In [4]:
from unfolded.data_sdk import DataSDK, MediaType

data_sdk = DataSDK(refresh_token="YOUR_REFRESH_TOKEN")

First we are going to add (upload) the dataset available in this directory. This is the same dataset used with our published map.

In [5]:
dataset = data_sdk.upload_file(
    file='earthquakes.geojson',
    name='Earthquakes data',
    media_type=MediaType.GEOJSON,
    description='USGS Earthquakes Oct/Nov 2020')

Then we can get the dataframe for the uploaded dataset and use it to create a Bokeh chart.

In [6]:
df = data_sdk.download_dataframe(dataset)
df.sample(3)

DriverError: Failed to read GeoJSON data

Here we are importing all of the [Bokeh library](https://docs.bokeh.org/en/latest/index.html) modules required to create our interactive chart:

In [7]:
from bokeh.io import show, output_notebook, reset_output
from bokeh.models import LinearColorMapper, HoverTool, ColumnDataSource
from bokeh.models.tools import WheelZoomTool
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.plotting import figure
reset_output()
output_notebook()

Loading BokehJS ...

Using the provided data we are creating an interactive vertical bar chart that displays earthquake magnitudes over time:

In [8]:
from datetime import timedelta

source = df
mags = source.mag.tolist()
times = source.time.tolist()
places = source.place.tolist()

cds = ColumnDataSource(data=dict(
    x=times,
    y=mags,
    desc=places,
))
TOOLTIPS = [
    ("Place", "@desc"),
    ("Date", "$x{%F}"),
    ("Magnitude", "$y"),
]

color_mapper = LinearColorMapper(palette='Magma256', low=min(mags), high=max(mags))

p = figure(title="Earthquake magnitudes over time", x_axis_type='datetime')
p.vbar(x='x', width=timedelta(hours=1), bottom=0, top='y', source=cds, color={'field': 'y', 'transform': color_mapper})

p.toolbar.active_scroll = p.select_one(WheelZoomTool)
p.xaxis.formatter=DatetimeTickFormatter(days=["%b %d, %Y"])

p.add_tools(HoverTool(tooltips=TOOLTIPS, formatters={'$x': 'datetime'}))

show(p)

NameError: name 'df' is not defined